# SQLAlchemy Unified Tutorial

https://docs.sqlalchemy.org/en/20/tutorial/index.html

In [1]:
# Version Check
import sqlalchemy

sqlalchemy.__version__

'2.0.30'

## The Engine

In [9]:
# Creating an "in-memory-only" SQLite DB instance
from sqlalchemy import create_engine

# The `echo` argument prints additional context from the DB to the python output
engine = create_engine("sqlite+pysqlite:///:memory:", echo=False)

SQLite uses "Lazy initialization" so even though the database has been created, it doesn't actually exist until an action is performed against it.

## Transactions and The DBAPI

In [10]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

[('hello world',)]


The above command was rolled back instead of committed, to commit data, you have to call `Connection.commit()`.

In [11]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )

    conn.commit()

Or, opening a context manager with engine.begin automatically commits the data, this is referred to as **begin once** style, if an exception occurs during the loop the transaction will be rolled back.

In [12]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

### Retrieving Data

In [27]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10


There are multiple ways to access the row objects:

**Tuple Assignment** - This is the most Python-idiomatic style, which is to assign variables to each row positionally as they are received:

In [28]:
with engine.begin() as conn:
    result = conn.execute(text("select x, y from some_table"))
    for x, y in result:
        print(f"x: {x}  y: {y}")

x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10


**Integer Index** - Tuples are Python sequences, so regular integer access is available too:

In [29]:
with engine.begin() as conn:
    result = conn.execute(text("select x, y from some_table"))
    for row in result:
        x = row[0]
        y = row[1]
        print(f"x: {x}  y: {y}")

x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10


**Attribute Name** - As these are Python named tuples, the tuples have dynamic attribute names matching the names of each column. These names are normally the names that the SQL statement assigns to the columns in each row. While they are usually fairly predictable and can also be controlled by labels, in less defined cases they may be subject to database-specific behaviors:

In [33]:
with engine.begin() as conn:
    result = conn.execute(text("select x, y from some_table"))

    for row in result:
        y = row.y
        # illustrate use with Python f-strings
        print(f"x: {row.x}  y: {y}")

x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10


**Mapping Access** - To receive rows as Python mapping objects, which is essentially a read-only version of Python’s interface to the common dict object, the Result may be transformed into a MappingResult object using the Result.mappings() modifier; this is a result object that yields dictionary-like RowMapping objects rather than Row objects:

In [34]:
with engine.begin() as conn:
    result = conn.execute(text("select x, y from some_table"))

    for dict_row in result.mappings():
        x = dict_row["x"]
        y = dict_row["y"]

        print(f"x: {x}  y: {y}")

x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10


### Sending Parameters

In [35]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

x: 2  y: 4
x: 6  y: 8
x: 9  y: 10


Note from SQLAlchemy documentation:

> As mentioned at the beginning of this section, textual SQL is not the usual way we work with SQLAlchemy. However, when using textual SQL, a Python literal value, even non-strings like integers or dates, should never be stringified into SQL string directly; a parameter should always be used. This is most famously known as how to avoid SQL injection attacks when the data is untrusted. However it also allows the SQLAlchemy dialects and/or DBAPI to correctly handle the incoming input for the backend. Outside of plain textual SQL use cases, SQLAlchemy’s Core Expression API otherwise ensures that Python literal values are passed as bound parameters where appropriate.

**Sending Multiple Parameters** - In the example at Committing Changes, we executed an INSERT statement where it appeared that we were able to INSERT multiple rows into the database at once. For DML statements such as “INSERT”, “UPDATE” and “DELETE”, we can send multiple parameter sets to the Connection.execute() method by passing a list of dictionaries instead of a single dictionary, which indicates that the single SQL statement should be invoked multiple times, once for each parameter set. This style of execution is known as executemany:

In [36]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

### Executing with an ORM Session

When the Session is used with non-ORM constructs, it passes through the SQL statements we give it and does not generally do things much differently from how the Connection does directly, so we can illustrate it here in terms of the simple textual SQL operations we’ve already learned.

The Session has a few different creational patterns, but here we will illustrate the most basic one that tracks exactly with how the Connection is used which is to construct it within a context manager:

In [37]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14


In [38]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}],
    )
    session.commit()

In [39]:
stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

x: 6  y: 8
x: 9  y: 11
x: 11  y: 12
x: 13  y: 15


## Database Metadata

### Setting up MetaData with Table objects

When we work with a relational database, the basic data-holding structure in the database which we query from is known as a table. In SQLAlchemy, the database “table” is ultimately represented by a Python object similarly named Table.

To start using the SQLAlchemy Expression Language, we will want to have Table objects constructed that represent all of the database tables we are interested in working with. The Table is constructed programmatically, either directly by using the Table constructor, or indirectly by using ORM Mapped classes (described later at Using ORM Declarative Forms to Define Table Metadata). There is also the option to load some or all table information from an existing database, called reflection.

Whichever kind of approach is used, we always start out with a collection that will be where we place our tables known as the MetaData object. This object is essentially a facade around a Python dictionary that stores a series of Table objects keyed to their string name. While the ORM provides some options on where to get this collection, we always have the option to simply make one directly, which looks like:

In [42]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [43]:
from sqlalchemy import Table, Column, Integer, String

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

### Components of Table

We can observe that the Table construct as written in Python has a resemblance to a SQL CREATE TABLE statement; starting with the table name, then listing out each column, where each column has a name and a datatype. The objects we use above are:

- **Table** - represents a database table and assigns itself to a MetaData collection.

- **Column** - represents a column in a database table, and assigns itself to a Table object. The Column usually includes a string name and a type object. The collection of Column objects in terms of the parent Table are typically accessed via an associative array located at Table.c:

In [44]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [45]:
user_table.c.keys()

['id', 'name', 'fullname']

- **Integer**, **String** - these classes represent SQL datatypes and can be passed to a Column with or without necessarily being instantiated. Above, we want to give a length of “30” to the “name” column, so we instantiated String(30). But for “id” and “fullname” we did not specify these, so we can send the class itself.

Additional resources:

The reference and API documentation for MetaData, Table and Column is at [Describing Databases with MetaData](https://docs.sqlalchemy.org/en/20/core/metadata.html). The reference documentation for datatypes is at [SQL Datatype Objects](https://docs.sqlalchemy.org/en/20/core/types.html).

### Declaring Simple Constraints

The first Column in the example user_table includes the Column.primary_key parameter which is a shorthand technique of indicating that this Column should be part of the primary key for this table. The primary key itself is normally declared implicitly and is represented by the PrimaryKeyConstraint construct, which we can see on the Table.primary_key attribute on the Table object:

In [46]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

The constraint that is most typically declared explicitly is the ForeignKeyConstraint object that corresponds to a database foreign key constraint. When we declare tables that are related to each other, SQLAlchemy uses the presence of these foreign key constraint declarations not only so that they are emitted within CREATE statements to the database, but also to assist in constructing SQL expressions.

A ForeignKeyConstraint that involves only a single column on the target table is typically declared using a column-level shorthand notation via the ForeignKey object. Below we declare a second table address that will have a foreign key constraint referring to the user table:

In [47]:
from sqlalchemy import ForeignKey

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

The table above also features a third kind of constraint, which in SQL is the “NOT NULL” constraint, indicated above using the Column.nullable parameter.

<div class="alert alert-block alert-info">
    <b>Tip:</b>
    When using the ForeignKey object within a Column definition, we can omit the datatype for that Column; it is automatically inferred from that of the related column, in the above example the Integer datatype of the user_account.id column.
</div>

### Emitting DDL to the Database

We’ve constructed an object structure that represents two database tables in a database, starting at the root MetaData object, then into two Table objects, each of which hold onto a collection of Column and Constraint objects. This object structure will be at the center of most operations we perform with both Core and ORM going forward.

The first useful thing we can do with this structure will be to emit CREATE TABLE statements, or DDL, to our SQLite database so that we can insert and query data from them. We have already all the tools needed to do so, by invoking the MetaData.create_all() method on our MetaData, sending it the Engine that refers to the target database:

In [48]:
metadata_obj.create_all(engine)

## Working with Data

## Data Manipulation with the ORM

## ORM Related Objects

## Further Reading